In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests
from hello import hello_logger
import os

In [2]:
def connect(ip, *args):
    global h
    h = HelloApp(ip, *args)
    
def dfmt(d):
    return d.strftime("%m/%d/%Y %I:%M:%S %p")
    
def wait1(s):
    if s <= 0: return
    dend = dfmt(datetime.datetime.now() + datetime.timedelta(seconds=s))
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait3(s, src):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=s)
    m1 = dfmt(start)
    m2 = dfmt(end)
    print("\r(%s) Sleeping %d seconds from %s to %s          " % (src, s, m1, m2), end="")
    tend = time.time() + s
    try:
        while time.time() < tend:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait2(s):
    pass
wait = wait1
    
def call(f, *a, _quiet=False, **k):
    if not _quiet:
        s = "(" + ", ".join(map(repr, a)) + ")"
        print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            return f(*a, **k)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv      

def setdo(mode, n2_or_sp, o2=None):
    call(h.setdo, mode, n2_or_sp,o2)
    
def doman(n2, o2):
    setdo(1, n2, o2)
    
def doauto(sp):
    setdo(0, sp)
    
def dooff():
    setdo(2, 0, 0)
    
def phoff():
    call(h.setph, 2, 0, 0)
        
def agauto(sp):
    call(h.setag, 0, sp)
    
    
def setmg(sp):
    call(h.setmg, 1, sp)

def pid1():
    call(h.setconfig, "DO", "O2_P_Gain_(%/DO%)", 4)
    call(h.setconfig, "DO", 'O2_I_Time_(min)', 55)
    call(h.setconfig, "DO", 'O2_D_Time_(min)', 0)
    call(h.setconfig, "DO", 'O2_Beta', 0)
    
    
    call(h.setconfig, "DO", "N2_P_Gain_(%/DO%)", -20)
    call(h.setconfig, "DO", 'N2_I_Time_(min)', 40)
    call(h.setconfig, "DO", 'N2_D_Time_(min)', 0)
    call(h.setconfig, "DO", 'N2_Beta', 0)
    
def docal1pt():
    v = call(h.getrawvalue, "doa")
    call(h.trycal, 'doa', v, 100)
    call(h.savetrialcal, 'doa')
    
def mk_pid(o2p, n2p, o2i, n2i):
    def pid():
        call(h.setconfig, "DO", "O2_P_Gain_(%/DO%)", o2p)
        call(h.setconfig, "DO", 'O2_I_Time_(min)', o2i)
        call(h.setconfig, "DO", 'O2_D_Time_(min)', 0)
        call(h.setconfig, "DO", 'O2_Beta', 0)

        call(h.setconfig, "DO", "N2_P_Gain_(%/DO%)", n2p)
        call(h.setconfig, "DO", 'N2_I_Time_(min)', n2i)
        call(h.setconfig, "DO", 'N2_D_Time_(min)', 0)
        call(h.setconfig, "DO", 'N2_Beta', 0)
    return pid

def mk_test(o2p, n2p, o2i, n2i):
    n = "%d.%d.%d.%d" % (o2p, n2p, o2i, n2i)
    p = mk_pid(o2p, n2p, o2i, n2i)
    return p, n

In [3]:
def setup_logger():
    global logger
    keys = [
        'temperature.pv',
        'agitation.pv',
        'ph.outputDown',
        'ph.manDown',
        'do.manUp',
        'do.outputDown',
        'do.pv',
        'do.manDown',
        'do.outputUp',
        'maingas.man',
        'maingas.mode',
        'maingas.pv',
        'MFCs.air',
        'MFCs.n2',
        'MFCs.o2',
        'MFCs.co2'
    ]
    try:
        logger
    except NameError:
        logger = None
    if logger is not None:
        logger.stop()
    logger = hello_logger.HelloLogger(h, 10, None, keys)
    logger.start()
    
def logfile(fn):
    print("Creating new file: %r"%fn)
    logger.new_file(fn)
    
def endlog():
    logger.new_file(None)
    
def connect(ip):
    global h
    h = HelloApp(ip)

In [4]:
# file save directory
os.chdir("C:\\users\\natha\\documents\\test")
d = "do_3L_verif_hs_new 170727"
os.makedirs(d, exist_ok=True)
os.chdir(d)

# stabilize at 100% DO 
def calibrate(w):
    dooff()
    agauto(18)
    setmg(0.5)
    wait(w)
    docal1pt()

# After running for 10 hours with pure headspace air,
# it should be safe to perform a 1 point cal (span)
# to ever so slightly increase accuracy.


def run_test(pid, name):

    pid()
    agauto(18)

    # 100% -> 150% test
    logfile("3L_do_pid %s 100 to 150.csv"%name)
    doauto(150)
    setmg(0.2)
    wait(3600*10)
    endlog()

    # ramp back down 150% -> 100% (zzz...)
    logfile("3L_do_pid %s 150 to 100.csv"%name)
    doauto(100)
    wait(3600*10)
    endlog()

    # Final test 100% -> 50%
    logfile("3L_do_pid %s 100 to 50.csv"%name)
    doauto(50)
    wait(3600*10)
    endlog()
    dooff()
    


In [5]:
connect("192.168.1.81")
wait = wait1
setup_logger()
calibrate(3600)

def test(a,b,c,d):
    p, n = mk_test(a,b,c,d)
    run_test(p, n)

test(4, -20, 55, 40)
test(20, -20, 50, 40)
test(3, -20, 60, 45)

dooff()
setmg(0.5)

Calling setdo(2, 0, 0)
Calling setag(0, 18)
Calling setmg(1, 0.5)
Calling getRawValue('doa')
Calling trycal('doa', 100.230469, 100)
Calling savetrialcal('doa')
Calling setconfig('DO', 'O2_P_Gain_(%/DO%)', 4)
Calling setconfig('DO', 'O2_I_Time_(min)', 55)
Calling setconfig('DO', 'O2_D_Time_(min)', 0)
Calling setconfig('DO', 'O2_Beta', 0)
Calling setconfig('DO', 'N2_P_Gain_(%/DO%)', -20)
Calling setconfig('DO', 'N2_I_Time_(min)', 40)
Calling setconfig('DO', 'N2_D_Time_(min)', 0)
Calling setconfig('DO', 'N2_Beta', 0)
Calling setag(0, 18)
Creating new file: '3L_do_pid 4.-20.55.40 100 to 150.csv'
Calling setdo(0, 150, None)
Calling setmg(1, 0.2)
Sleeping 36000 seconds, ending at 07/21/2017 10:55:56 PM
Creating new file: '3L_do_pid 4.-20.55.40 150 to 100.csv'
Calling setdo(0, 100, None)
Sleeping 36000 seconds, ending at 07/22/2017 08:55:59 AM
Creating new file: '3L_do_pid 4.-20.55.40 100 to 50.csv'
Calling setdo(0, 50, None)
Sleeping 36000 seconds, ending at 07/22/2017 06:56:02 PM
Calling se

In [6]:
# Hello server crapped out and swallowed the call to stabilize DO PV at 100% for the 100-> 50%
# test, so it needs to be repeated. 

connect("192.168.1.81")
wait = wait1
setup_logger()
calibrate(30)

def run_test2(pid, name):
    pid()
    agauto(18)

    # Final test 100% -> 50%
    logfile("3L_do_pid %s 100 to 50.csv"%name)
    doauto(50)
    wait(3600*10)
    endlog()
    dooff()

def test2(a,b,c,d):
    p, n = mk_test(a,b,c,d)
    run_test2(p, n+" repeat")

test2(4, -20, 55, 40)

dooff()
setmg(0.5)

Calling setdo(2, 0, 0)
Calling setag(0, 18)
Calling setmg(1, 0.5)
Sleeping 30 seconds, ending at 07/27/2017 03:44:31 PM
Calling getRawValue('doa')
Calling trycal('doa', 100.026581, 100)
Calling savetrialcal('doa')
Calling setconfig('DO', 'O2_P_Gain_(%/DO%)', 4)
Calling setconfig('DO', 'O2_I_Time_(min)', 55)
Calling setconfig('DO', 'O2_D_Time_(min)', 0)
Calling setconfig('DO', 'O2_Beta', 0)
Calling setconfig('DO', 'N2_P_Gain_(%/DO%)', -20)
Calling setconfig('DO', 'N2_I_Time_(min)', 40)
Calling setconfig('DO', 'N2_D_Time_(min)', 0)
Calling setconfig('DO', 'N2_Beta', 0)
Calling setag(0, 18)
Creating new file: '3L_do_pid 4.-20.55.40 repeat 100 to 50.csv'
Calling setdo(0, 50, None)
Sleeping 36000 seconds, ending at 07/28/2017 01:44:36 AM
Calling setdo(2, 0, 0)
Calling setdo(2, 0, 0)
Calling setmg(1, 0.5)


# 3L Analysis

In [ ]:
h = HelloApp('71.189.82.196:81')
p='C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\PID Tuning\\DO\\Raw Data\\'

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn):
    if isinstance(bn, str):
        b = call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = call(h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+"3L " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [ ]:
for s in (" (continued01)", 2,3):
    dl("dopid test%s"%s)

In [7]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
    
def analyze(fn):
    c1 = cells.Find("DOPV(%)")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "24"
    mkch(c1,"DOPV(%)")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    
    for v in "DON2FlowActualRequest(%)", "DOO2FlowControllerRequestLimited(%)":
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        mkser(ch2, c, v)
    fixaxes()
    wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)
        
def analf(f):
    open_file(f)
    analyze(f)

In [11]:
f = '3L_do_pid 4.-20.55.40 repeat 100 to 50.csv'
analf(os.path.abspath(f))

AttributeError: 'NoneType' object has no attribute 'Offset'

Traceback (most recent call last):
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\program files\python 3.5\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\program files\python 3.5\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connectionpool.py", line 345, in _make_reque

Traceback (most recent call last):
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\program files\python 3.5\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\program files\python 3.5\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connectionpool.py", line 345, in _make_reque

Traceback (most recent call last):
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\program files\python 3.5\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\program files\python 3.5\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "c:\program files\python 3.5\lib\site-packages\urllib3\connectionpool.py", line 345, in _make_reque

Traceback (most recent call last):
  File "C:\Users\natha\Documents\Dropbox\Python\hello\hello_logger.py", line 285, in run
    self.fn()
  File "C:\Users\natha\Documents\Dropbox\Python\hello\hello_logger.py", line 176, in log_data
    d = self._query()
  File "C:\Users\natha\Documents\Dropbox\Python\hello\hello_logger.py", line 161, in _query
    mv = self._gmv()
  File "C:\Users\natha\Documents\Dropbox\Python\hello\hello_logger.py", line 158, in _gmv
    return self._call(self._h.gpmv)
  File "C:\Users\natha\Documents\Dropbox\Python\hello\hello_logger.py", line 142, in _call
    return f(*args, **kw)
  File "C:\Users\natha\Documents\Dropbox\Python\hello\hello3.py", line 585, in getMainValues
    mv = json_loads(rsp.read().decode('utf-8'))
  File "c:\program files\python 3.5\lib\json\__init__.py", line 319, in loads
    return _default_decoder.decode(s)
  File "c:\program files\python 3.5\lib\json\decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
 